In [4]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import astropy
from astropy import constants as ac
import lightkurve as lk
from scipy.optimize import minimize
from astropy.io import fits
from astropy.table import Table
from lightkurve import search_lightcurvefile
from lightkurve import search_lightcurve
from lightkurve import search_targetpixelfile
import exoplanet as xo

In [5]:
from matplotlib import rcParams
rcParams["savefig.dpi"] = 100
rcParams["figure.dpi"] = 100
rcParams["font.size"] = 18
rcParams["figure.figsize"] = [8, 5]
rcParams["font.family"] = "sans-serif"
rcParams["font.sans-serif"] = ["Computer Modern Sans Serif"]
rcParams["text.usetex"] = True
plt.rcParams['axes.linewidth']=1.5

In [6]:
jr = pd.read_csv('/Users/lupitatovar/Cool-Worlds/kois_JR.csv')
jr

,KIC,KOI,Kepler-ID,TTVperiod,TTVperiod_e,Epoch,Epoch_e,RpRs,RpRs_p,RpRs_m,...,[M/H]_e,StatusFlag,rhostar_model_lc,Epoch_lc,Per_lc,b_lc,RpRs_lc,lcflag,ttvper_cor,ttvper_cor_err
0,11446443,1.01,Kepler-1 b,2.470613,2.000000e-08,787.064865,0.000009,0.123869,0.000066,-0.000062,...,0.041,PPPP,1.83550,55.763305,2.470613,0.8178,0.123864,0,0.000000e+00,1.000000e-08
1,10666592,2.01,Kepler-2 b,2.204735,5.000000e-08,788.535431,0.000017,0.075184,0.000008,-0.000009,...,0.042,PPPP,0.40817,54.358544,2.204735,0.0005,0.075181,0,2.000000e-08,2.000000e-08
2,10748390,3.01,Kepler-3 b,4.887803,2.400000e-07,786.095759,0.000082,0.057856,0.000471,-0.000123,...,0.084,PPPP,3.65395,57.813080,4.887803,0.1028,0.057902,0,2.000000e-08,5.000000e-08
3,3861595,4.01,Kepler-1658 b,3.849371,1.160000e-06,787.262929,0.000525,0.039553,0.000820,-0.001229,...,0.082,PPPP,0.21898,90.526690,3.849372,0.9165,0.039657,0,-1.100000e-07,3.000000e-07
4,8554498,5.01,NaN,4.780328,8.700000e-07,787.803519,0.000159,0.036536,0.000218,-0.000187,...,0.158,PPPP,0.34945,65.974079,4.780328,0.9512,0.036532,0,-1.000000e-08,1.800000e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9718,11610352,8420.01,NaN,4.377549,2.119800e-04,790.506658,0.021818,0.007583,0.001149,-0.001149,...,0.140,SNNN,1.08218,68.230876,4.377417,0.0659,0.007212,0,5.790000e-06,4.843000e-05
9719,11653065,8421.01,NaN,0.284970,9.300000e-07,794.013069,0.004723,0.006505,0.000606,-0.000371,...,0.151,PNNN,0.43429,64.766425,0.284974,0.4890,0.006649,0,-1.120000e-05,3.260000e-06
9720,11867853,8422.01,NaN,0.642227,3.070000e-06,794.122100,0.007560,0.007498,0.003033,-0.001300,...,0.141,SNNN,1.09801,65.175282,0.642237,0.3420,0.007224,0,-6.440000e-06,4.780000e-06
9721,12069878,8423.01,NaN,42.038145,2.602350e-03,753.695994,0.011121,0.010799,0.001395,-0.001014,...,0.129,SNNN,23.73683,81.091603,42.037535,0.2816,0.010708,0,7.540000e-06,6.191000e-05


In [11]:
kois = pd.read_csv('/Users/lupitatovar/Cool-Worlds/targets/candidates_only_kois.csv')
kois

,name,KIC,Insolation,Period,T_eff,Radius,masse_est,semi_au,st_log_lum,Discovery,H_Tidal,H_Radio,H_Total,TSM,Contrast,Theta
0,KOI-4427.01,4172805,0.318849,147.661729,199.239771,1.797673,3.885904,0.447412,-1.195000,Kepler,4.269050e+10,3.370000e+14,3.370000e+14,2.627650,7.260000e-09,0.001642
1,KOI-4848.01,9005321,0.160519,384.696100,167.826873,1.880000,4.192848,0.903101,-0.883000,Kepler,2.197108e+10,3.860000e+14,3.860000e+14,1.401344,1.950000e-09,0.001913
2,KOI-7716.01,5097856,0.434274,483.364050,215.238882,1.503482,2.868943,1.153766,-0.238000,Kepler,4.704759e+08,1.970000e+14,1.970000e+14,0.839124,7.640000e-10,0.001501
3,KOI-5087.01,4770798,0.332875,651.074310,201.395598,1.141911,1.563745,1.441648,-0.160000,Kepler,1.584473e+07,8.650000e+13,8.650000e+13,1.533628,2.820000e-10,0.004759
4,KOI-5227.01,6034834,0.598982,371.564920,233.256197,1.611944,3.229113,0.967816,-0.251000,Kepler,2.518710e+09,2.430000e+14,2.430000e+14,1.520763,1.250000e-09,0.001904
5,KOI-5236.01,6067545,0.811569,550.865140,251.658130,1.921435,4.350944,1.334558,0.160000,Kepler,6.994064e+08,4.120000e+14,4.120000e+14,1.883685,9.330000e-10,0.002571
6,KOI-5130.01,5166811,0.430720,370.059180,214.797110,1.900000,4.268858,0.938774,-0.420683,Kepler,1.127595e+09,3.980000e+14,3.980000e+14,1.134144,1.840000e-09,0.001022
7,KOI-5408.01,7631138,0.354582,415.748785,204.601495,1.710821,3.572556,1.034289,-0.421000,Kepler,3.826753e+08,2.910000e+14,2.910000e+14,1.659374,1.230000e-09,0.002373
8,KOI-5789.01,10387792,0.319386,383.262170,199.323573,1.532865,2.964783,0.944805,-0.545000,Kepler,5.560228e+08,2.090000e+14,2.090000e+14,1.655275,1.180000e-09,0.002474
9,KOI-5798.01,10487228,0.403587,318.258580,211.331357,1.963136,4.512475,0.834702,-0.551000,Kepler,2.157382e+09,4.390000e+14,4.390000e+14,1.181350,2.490000e-09,0.001076


Cross matching my list of Kepler candidates with Jason Rowe's list of KOIs to compare values/dispositions etc..


In [12]:
jr['KIC'].values

array([11446443, 10666592, 10748390, ..., 11867853, 12069878, 12070798])

In [13]:
kois['KIC'].values

array([ 4172805,  9005321,  5097856,  4770798,  6034834,  6067545,
        5166811,  7631138, 10387792, 10487228,  8656535, 10581308,
        9079671,  9114614, 10007916,  8352009,  7416016,  7416646,
        9220614, 12007270, 12009917, 12418418, 12734110, 11129145,
        9221517, 11671646,  3548622,  8297319,  9084569, 11294822,
        9650579])

In [14]:
len(kois)

31

In [20]:
crossmatch = kois.merge(jr, how="inner",on=['KIC'])

In [21]:
#crossmatch.to_csv('/Users/lupitatovar/Cool-Worlds/targets/jr_koi_xmatch.csv')

In [23]:
crossmatch.columns

Index(['name', 'KIC', 'Insolation', 'Period', 'T_eff', 'Radius', 'masse_est',
       'semi_au', 'st_log_lum', 'Discovery', 'H_Tidal', 'H_Radio', 'H_Total',
       'TSM', 'Contrast', 'Theta', 'KOI', 'Kepler-ID', 'TTVperiod',
       'TTVperiod_e', 'Epoch', 'Epoch_e', 'RpRs', 'RpRs_p', 'RpRs_m', 'b',
       'b_ep', 'b_em', 'rhostar_model', 'rhostar_model_ep', 'rhostar_model_em',
       'u1', 'u2', 'nTTobs', 'nTT', 'TTVflag', 'tdepth', 'tdepth_e', 'tdur',
       'tdur_e', 'tduravg', 'tduravg_e', 'radius', 'radius_ep', 'radius_em',
       'SNR', 'SNRwoTTV-SNRwTTV', 'MES', 'chisqwttv', 'chisqwottv',
       'dchisq_inj', 'adrs', 'adrs_ep', 'adrs_em', 'incl', 'incl_ep',
       'incl_em', 's0', 's0_ep', 's0_em', 'kepmag', 'stellar_source',
       'rhostar', 'rhostar_ep', 'rhostar_em', 'teff', 'teff_e', 'rstar',
       'rstar_ep', 'rstar_em', 'mstar', 'mstar_ep', 'mstar_em', 'logg',
       'logg_ep', 'logg_em', '[M/H]', '[M/H]_e', 'StatusFlag',
       'rhostar_model_lc', 'Epoch_lc', 'Per_lc', 'b

In [ ]:
#which column is the disposition???

In [28]:
crossmatch['Radius']

0     1.797673
1     1.880000
2     1.503482
3     1.141911
4     1.611944
5     1.921435
6     1.900000
7     1.710821
8     1.532865
9     1.963136
10    1.608549
11    1.395906
12    1.489459
13    1.894865
14    1.503236
15    1.992590
16    1.352441
17    1.581144
18    1.578987
19    1.398538
20    1.717552
21    1.080000
22    1.780000
23    0.886233
24    1.485403
25    1.485403
26    1.575000
27    1.960000
28    1.274701
29    0.910840
30    1.862796
31    1.628521
Name: Radius, dtype: float64

In [27]:
crossmatch['radius']

0     1.818
1     1.663
2     1.408
3     1.173
4     1.451
5     1.943
6     1.988
7     1.610
8     1.468
9     1.728
10    1.403
11    1.222
12    1.012
13    1.763
14    1.422
15    1.730
16    1.092
17    1.329
18    1.526
19    1.288
20    1.668
21    1.371
22    1.812
23    0.856
24    0.815
25    0.957
26    1.205
27    1.843
28    1.242
29    0.896
30    1.851
31    1.752
Name: radius, dtype: float64

In [29]:
crossmatch['StatusFlag'].values

array(['PPPP', 'PPNN', 'PPPN', 'PPNN', 'PPNN', 'PPFP', 'PPNN', 'SPNN',
       'PPNN', 'SPNN', 'PPNN', 'SPNN', 'SPNN', 'PPNN', 'PPNN', 'PPNN',
       'SPNN', 'SPNN', 'SPNN', 'SPNN', 'SPNN', 'PPNN', 'SPNN', 'SPNN',
       'PPPP', 'SPNN', 'PPNP', 'PPFP', 'PPNP', 'PPPN', 'PPPN', 'PPPN'],
      dtype=object)